In [1]:
import requests
import pandas as pd

/var/folders/yh/tjfmc6td3jd69r72yb6mpcvh0000gq/T/ipykernel_98361/222825045.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# acquire data

In [22]:
# note that this had to strip the last two parameters from the api url in the google doc
# also upped the limit 
api_url = "https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0 "
response = requests.get(api_url)
data = response.json()

In [23]:
data

[{'sno': '1',
  'complaint': '222010',
  'complaint_description': 'Black and gray smoke coming from Wheelabrator Stack',
  'complaint_type': 'Smoke',
  'recieved_date': '2024-01-03',
  'incident_date': '2024-01-03',
  'incident_closed_date': '2024-01-24',
  'incident_status_desc': 'Incident Closed-Managed',
  'incident_zip': '21230'},
 {'sno': '2',
  'complaint': '222009',
  'complaint_description': 'Smoke coming out of the building',
  'complaint_type': 'Smoke',
  'recieved_date': '2024-01-24',
  'incident_date': '2024-01-24',
  'incident_status_desc': 'Under Investigation',
  'incident_zip': '21226'},
 {'sno': '3',
  'complaint': '222008',
  'complaint_description': 'Air smelt like burning rubber.  Smoke coming from Curtis Bay Medical waste.',
  'complaint_type': 'Odor, Smoke',
  'recieved_date': '2024-01-16',
  'incident_date': '2024-01-13',
  'incident_status_desc': 'Under Investigation',
  'incident_zip': '21226'},
 {'sno': '4',
  'complaint': '222007',
  'complaint_description': 

In [20]:
df = pd.DataFrame(data)

In [21]:
df

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,incident_closed_date,incident_status_desc,incident_zip,county
0,1,222010,Black and gray smoke coming from Wheelabrator ...,Smoke,2024-01-03,2024-01-03,2024-01-24,Incident Closed-Managed,21230,NaN
1,2,222009,Smoke coming out of the building,Smoke,2024-01-24,2024-01-24,NaN,Under Investigation,21226,NaN
2,3,222008,Air smelt like burning rubber. Smoke coming f...,"Odor, Smoke",2024-01-16,2024-01-13,NaN,Under Investigation,21226,NaN
3,4,222007,Plumes of black smoke coming from the facility.,Smoke,2024-01-05,2024-01-05,NaN,Under Investigation,21226,NaN
4,5,222005,Dust from concrete grinding,Fugitive Dust/Particulate Matter,2024-01-17,NaN,2024-01-24,Incident Closed-Managed,21227,Baltimore
...,...,...,...,...,...,...,...,...,...,...
1425,1426,8058,Boat company sandblasting boats and blowing wh...,Fugitive Dust/Particulate Matter,2021-01-11,2021-01-11,2022-03-08,Incident Closed - No further action,NaN,Queen Anne's
1426,1427,8057,Smoke and ash from neighbor's open burning in ...,Air,2021-01-07,2021-01-07,2021-01-07,Incident Closed-No Violation Observed,20751,Anne Arundel
1427,1428,8056,Smoke from neighbor's wood burning stove.,Air,2021-01-07,2021-01-04,NaN,Under Investigation,NaN,Charles
1428,1429,8051,Concern of materials being burned in fireplace...,Air,2021-01-04,2020-12-28,2021-01-15,Incident Closed-No Violation Observed,21770,Frederick


# clean data

# analyze data

# export data